In [ ]:
import os
from Bio import SeqIO
import time
import matplotlib.pyplot
import numpy as np
import math

# Getting started
**NOTE**: we assume that the groundtruths are known since we base our results on simulations. In this, we assume that the user has access to a pair of files for every sample called `sample_sampleid_groundtruth_reads.tsv` and `sample_sampleid_groundtruth_folds` located in the corresponding sample files. These files are provided on [Zenodo](https://doi.org/10.5281/zenodo.14727633) for the experiments in our manuscript and have for every line a species taxid followed by the relative abundance (either read-based or fold-based), delimited by a tab. For the read-based groundtruths a third column is added with the absolute number of reads of every species.

In [ ]:
def read_groundtruth_reads(sample_number):
    abundances = {}
    read_totals = {}
    with open(f"samples/sample_{sample_number}/sample_{sample_number}_groundtruth_reads.tsv", "r") as f_in:
        for line in f_in:
            line = line.strip().split("\t")
            abundances[int(line[0])] = float(line[1])
            read_totals[int(line[0])] = int(line[2])
    return abundances, read_totals

def read_groundtruth_folds(sample_number):
    abundances = {}
    with open(f"samples/sample_{sample_number}/sample_{sample_number}_groundtruth_folds.tsv", "r") as f_in:
        for line in f_in:
            line = line.strip().split("\t")
            abundances[int(line[0])] = float(line[1])
    return abundances

# Gather groundtruth for example sample_1
groundtruth_reads, groundtruth_totals = read_groundtruth_reads(1)
groundtruth_folds = read_groundtruth_folds(1)

## Bracken

In [ ]:
def process_bracken()